In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.head()

In [ ]:
df['Churn'] = df['Churn'].map({'No': 0,
                            'Yes': 1})

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.drop(['customerID'], axis = 1, inplace = True)

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
X_temp = df.copy()

encoder = LabelEncoder()

for column in X_temp.select_dtypes(include = ['object']).columns:
    X_temp[column] = encoder.fit_transform(X_temp[column])

X = X_temp.drop(['Churn'], axis = 1)
y = X_temp['Churn']

# RandomForest and permutation_importance
clf = RandomForestClassifier(random_state=42)
clf.fit(X, y)

result = permutation_importance(clf, X, y, n_jobs=1, random_state=0)
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': result.importances_mean})

feature_importance_df.sort_values(by='Importance', ascending=False)

In [ ]:
plt.figure(figsize=(10, 5), dpi=96)
df['Churn'].value_counts().plot(kind = 'barh')
plt.title("Frequency of the churn")
plt.show()

In [ ]:
label_encoder = LabelEncoder()
for column in df.select_dtypes(include=['object']).columns:
    df[column] = label_encoder.fit_transform(df[column])
X = df.drop(['Churn'], axis=1)
y = df['Churn']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

In [ ]:
smote = SMOTE(random_state = 42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

In [ ]:
y_resampled.value_counts()

In [ ]:
#SVM Model
svc_model = SVC(random_state = 30)
svc_model.fit(X_resampled, y_resampled)

In [ ]:
svc_model.score(X_resampled, y_resampled)*100

In [ ]:
y_test_pred_svm = svc_model.predict(X_test)
confusion_matrix(y_test, y_test_pred_svm)

In [ ]:
print(classification_report(y_test, y_test_pred_svm))

In [ ]:
#Random Forest Classifier
rf_clf = RandomForestClassifier(random_state = 42)
rf_clf.fit(X_resampled, y_resampled)

In [ ]:
rf_clf.score(X_resampled, y_resampled)*100

In [ ]:
y_test_pred_rfc = rf_clf.predict(X_test)

In [ ]:
confusion_matrix(y_test, y_test_pred_rfc)

In [ ]:
print(classification_report(y_test, y_test_pred_rfc))